In [4]:
import os
import pandas as pd
df = pd.read_csv('../data/raw/olist_orders_dataset.csv')

In [ ]:
print(f"order_status: {df['order_status'].nunique()} unique values")
print(df['order_status'].value_counts())

order_status: 8 unique values
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64


In [14]:
date_cols = [c for c in df.columns if ('date' in c.lower() or 'timestamp' in c.lower() or c.endswith('_at'))]
# ensure they're datetimes (bad parses become NaT)
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

miss_pct = df.groupby('order_status')[date_cols].apply(lambda x: x.isna().mean() * 100)
print("Missingness (%) by order_status:")
print(miss_pct)

Missingness (%) by order_status:
              order_purchase_timestamp  order_approved_at  \
order_status                                                
approved                           0.0           0.000000   
canceled                           0.0          22.560000   
created                            0.0         100.000000   
delivered                          0.0           0.014511   
invoiced                           0.0           0.000000   
processing                         0.0           0.000000   
shipped                            0.0           0.000000   
unavailable                        0.0           0.000000   

              order_delivered_carrier_date  order_delivered_customer_date  \
order_status                                                                
approved                        100.000000                     100.000000   
canceled                         88.000000                      99.040000   
created                         100.000000      

In [18]:
# outlier / missing analysis
df[(df['order_status'] == 'delivered') & (df['order_delivered_customer_date'].isna())]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,2017-11-28 17:56:40,2017-11-30 18:12:23,NaT,2017-12-18
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,2018-06-20 07:19:05,2018-06-25 08:05:00,NaT,2018-07-16
43834,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,2018-07-01 17:15:12,2018-07-03 13:57:00,NaT,2018-07-30
79263,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,2018-07-01 22:15:14,2018-07-03 13:57:00,NaT,2018-07-30
82868,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,2018-07-01 21:29:54,2018-07-03 09:28:00,NaT,2018-07-24
92643,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,2017-05-25 23:30:16,NaT,NaT,2017-06-23
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,2018-06-08 12:36:39,2018-06-12 14:10:00,NaT,2018-06-26
98038,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,2018-06-27 16:29:30,2018-07-03 19:26:00,NaT,2018-07-19


In [20]:
# load onto pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean
spark = SparkSession.builder.appName("OlistOrders").getOrCreate()


sdf = spark.read.csv('../data/raw/olist_orders_dataset.csv', header=True, inferSchema=True)
sdf.printSchema()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is not supported
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:277)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
	at py4j.reflection.Metho